In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import os
# from torch.utils.data import DataLoader
# from torchvision import datasets, transforms
import torch.nn.functional as F

from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from sklearn.model_selection import train_test_split
from utils import *


In [2]:
# class Net(nn.Module):
#     def __init__(self):
#         super().__init__() # 7 x 22 x 22
#         self.conv1 = nn.Conv2d(7, 16, 4,stride=2) # 16 x 10 x 10
#         self.pool = nn.MaxPool2d(2,stride=1) # 16 x 9 x 9
#         self.conv2 = nn.Conv2d(16, 32, 3,stride=2) # 32 x 4 x 4
#         self.drop = nn.Dropout(p=0.15)
#         self.fc1 = nn.Linear( 512, 120)   
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 2)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = F.relu(self.conv2(x))
#         x = torch.flatten(x,1) 
#         x = self.drop(x)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = F.softmax(self.fc3(x), dim=1)
#         return x
    
# def retrieve_images(fil_dir):
#     sample_list = os.listdir(fil_dir)
#     img_list = np.zeros((len(sample_list),7,40,500))
#     ind = 0
#     for file_name in sample_list:
#         img_list[ind,:,:,:] = np.load(os.path.join(fil_dir,file_name))
#         ind+=1
#     return img_list[:ind,:,:,:]

# def retrieve_masks(fil_dir):
#     sample_list = os.listdir(fil_dir)
#     img_list = np.zeros((len(sample_list),40,500))
#     ind = 0
#     for file_name in sample_list:
#         img_list[ind,:,:] = np.load(os.path.join(fil_dir,file_name))
#         ind+=1
#     return img_list.reshape(-1,1,40,500)[:ind,:,:,:]

# def get_windowed_data(imgs, masks):
#     windows_per_im = 10*240
#     fin_data = np.zeros((imgs.shape[0]*windows_per_im,7,22,22))
#     fin_labs = np.zeros((imgs.shape[0]*windows_per_im,1))
#     for ind in range(imgs.shape[0]):
#         fin_data[ind*windows_per_im:(ind+1)*windows_per_im,:,:,:] = sliding_window(imgs[ind,:,:,:])
#         mask_windows = sliding_window(masks[ind,:,:,:])
#         for el in range(mask_windows.shape[0]):
#             if np.amax(mask_windows[el])>0:
#                 fin_labs[ind*windows_per_im+el]=1
#     return fin_data, fin_labs

# def sliding_window(image, sq_size = 22, pix_steps=2):
#     img_list = []
#     for i in np.arange(0,image.shape[1]-sq_size+pix_steps,pix_steps):
#         for j in np.arange(0,image.shape[2]-sq_size+pix_steps,pix_steps):
#             img_list.append(image[:,i:i+sq_size,j:j+sq_size])
#     return np.array(img_list)


In [3]:
net = Net_result()
net.load_state_dict(torch.load("C:/Users/laure/Documents/proj_loopr/models_2_21_6_dropout_43tp_93tn/trained_net_70_epo.pth"))
net.eval()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/laure/Documents/proj_loopr/models_2_21_6_dropout_43tp_93tn/trained_net_70_epo.pth'

In [4]:
img_list = retrieve_images("C:/Users/laure/Documents/proj_loopr/processed_images")
masks = retrieve_masks("C:/Users/laure/Documents/proj_loopr/all_masks")
X_test, y_test = get_windowed_data(img_list,masks)
# name_list = sorted(os.listdir("C:/Users/laure/Documents/proj_loopr/processed_images"))
# ys = [1. if el[5:8]=="000" else 0. for el in name_list]
ys = np.array(y_test).reshape((-1,1))
ys = np.hstack(( 1-ys,ys))

In [5]:
test_dataset = TensorDataset(torch.from_numpy(X_test).type(torch.FloatTensor), torch.tensor(ys))
test_loader = DataLoader(test_dataset, batch_size=50)

num_correct = 0
true_neg = 0
with torch.no_grad():
    for batch, (val_inputs, val_labs) in enumerate(test_loader):
        val_outputs = net(val_inputs).detach().numpy()
        val_outputs = (val_outputs[:,1]>val_outputs[:,0]).astype(int)
        num_correct += np.sum(val_labs.numpy()[:,1].flatten() * val_outputs.flatten())
        true_neg += np.sum(val_labs.numpy()[:,0].flatten() * (1-val_outputs.flatten()))
        # num_correct+=np.argwhere([:,1] ==val_labs.numpy()[:,1]).flatten().shape[0]
acc = num_correct / np.sum(ys[:,1])
tn_rate = true_neg / np.sum(ys[:,0])
print("Sensitivity: "+str(acc))
print("Specificity: "+str(tn_rate))

Sensitivity: 0.6831651754552954
Specificity: 0.9479869495544709


In [4]:
out_model = Net_result()
out_model.load_state_dict(torch.load("C:/Users/laure/Documents/proj_loopr/result_model_2_24/trained_net_50_epo.pth"))
out_model.eval()

Net_result(
  (conv1): Conv2d(84, 32, kernel_size=(1, 2), stride=(1, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 16, kernel_size=(2, 2), stride=(1, 2))
  (drop): Dropout(p=0.15, inplace=False)
  (fc1): Linear(in_features=1920, out_features=360, bias=True)
  (fc2): Linear(in_features=360, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=2, bias=True)
)

In [5]:
source_dir = "C:/Users/laure/Documents/proj_loopr/windowed_model_outputs_2_24_57tp_90tn"
xs = retrieve_images(source_dir, shape=(84,10,240))
name_list = sorted(os.listdir(source_dir))
ys = [1. if el[5:8]=="000" else 0. for el in name_list]
ys = np.array(ys).reshape((-1,1))
ys = np.hstack((ys, 1-ys))

In [6]:
final_test(out_model, xs,ys)

Sensitivity: 0.8653846153846154
Specificity: 0.9929078014184397


### 94% accuracy total